In [1]:
from PIL import Image
import numpy as np 
import matplotlib.pyplot as plt
import math

In [2]:
def get_filter_size(sigma):
    s = (2 * math.ceil(3*sigma)) + 1
    return s

In [3]:
def calculate_log(x, y, sigma):
    return (-1/(math.pi*(sigma**4))) * (1 - ( ((x**2)+(y**2))/(2*(sigma**2)) ) ) * (math.e**( ((x**2)+(y**2)) / (2*(sigma**2)) ))

In [8]:
def calculate_filter(sigma):
    size = get_filter_size(sigma)
    log_filter = np.zeros((size,size))
    
    center_x = math.floor(size / 2)
    center_y = math.floor(size / 2)
    
    for i in range(size):
        for j in range(size):
            distance_x = abs(center_x - j)
            distance_y = abs(center_y - i)
            log_filter[i][j] = calculate_log(distance_x,distance_y,sigma)
            
    return log_filter, size

In [6]:
def detect_edge(img, sigma):
    original_img = img.copy()
    log_filter, size = calculate_filter(sigma)
    
    # Apply "log_filter" on "original_img" in some way
    # and put the values in "img"

In [9]:
img = Image.open("Cameraman.tif")

img_result = img.copy()
detect_edge(img_result, 2)
plt.imsave("LoG_2.jpg", img_result, cmap='gray')
plt.imshow(img_result, cmap="gray")

img_result = img.copy()
detect_edge(img_result, 3)
plt.imsave("LoG_3.jpg", img_result, cmap='gray')
plt.imshow(img_result, cmap="gray")

img_result = img.copy()
detect_edge(img_result, 4)
plt.imsave("LoG_4.jpg", img_result, cmap='gray')
plt.imshow(img_result, cmap="gray")

13
[[ 1.28964586e+03  2.70029649e+02  7.27791479e+01  2.55122254e+01
   1.18103439e+01  7.35614251e+00  6.26791874e+00  7.35614251e+00
   1.18103439e+01  2.55122254e+01  7.27791479e+01  2.70029649e+02
   1.28964586e+03]
 [ 2.70029649e+02  5.41040729e+01  1.38010878e+01  4.53278431e+00
   1.95964295e+00  1.15443565e+00  9.62186668e-01  1.15443565e+00
   1.95964295e+00  4.53278431e+00  1.38010878e+01  5.41040729e+01
   2.70029649e+02]
 [ 7.27791479e+01  1.38010878e+01  3.25858705e+00  9.62186668e-01
   3.63544525e-01  1.87395191e-01  1.47000600e-01  1.87395191e-01
   3.63544525e-01  9.62186668e-01  3.25858705e+00  1.38010878e+01
   7.27791479e+01]
 [ 2.55122254e+01  4.53278431e+00  9.62186668e-01  2.35940634e-01
   6.31449604e-02  1.73595417e-02  7.65987090e-03  1.73595417e-02
   6.31449604e-02  2.35940634e-01  9.62186668e-01  4.53278431e+00
   2.55122254e+01]
 [ 1.18103439e+01  1.95964295e+00  3.63544525e-01  6.31449604e-02
  -0.00000000e+00 -1.39378396e-02 -1.64001338e-02 -1.39378396e-